In [ ]:
%load_ext autoreload
%autoreload 2

def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))

In [ ]:
from supclr_train import dataio_ssl_prep
import torch
import speechbrain as sb

In [ ]:
hparams = {
    'sample_rate': 16000,
    'train_duration': 8.0
}
label_encoder_path = './dataset/label_encoder_tauuas2019_ordered.txt'
label_encoder = sb.dataio.encoder.CategoricalEncoder()
label_encoder.load_or_create(label_encoder_path)

csv_path = '/mnt/data1/zhepei/outputs/cssl_sound/split_test/debug_2022-03-26+23-09-23_seed_2022+ssl_tau//save/train_task0_replay.csv'
ds = dataio_ssl_prep(
    hparams,
    csv_path,
    label_encoder
)

In [ ]:
idx = torch.randint(0, len(ds), (1,)).item()
x1 = ds[idx]['sig1']
x2 = ds[idx]['sig2']
class_name = ds[idx]['class_name']

sound(x1.numpy(), 16000, label='{}_{}_x1'.format(idx, class_name))
sound(x2.numpy(), 16000, label='{}_{}_x2'.format(idx, class_name))

# Time Domain Augmentation

In [ ]:
# from speechbrain.lobes import augment
# time_domain_aug = augment.TimeDomainSpecAugment(
#     sample_rate=hparams['sample_rate'],
#     speeds=[95, 100, 105],
#     drop_freq_count_high=4,
#     drop_chunk_count_high=3,
#     drop_chunk_length_low=1000,
#     drop_chunk_length_high=8000,
#     drop_chunk_noise_factor=0.,
# #     drop_chunk_count_high=2,
# #     drop_chunk_noise_factor=0.05,
# )

In [ ]:
# def compute_aug(wavs):
#     lens = torch.ones(wavs.shape[0]).to(wavs.device)
#     wavs_aug = time_domain_aug(wavs, lens)
#     if wavs_aug.shape[1] > wavs.shape[1]:
#         wavs_aug = wavs_aug[:, 0 : wavs.shape[1]]
#     else:
#         zero_sig = torch.zeros_like(wavs)
#         zero_sig[:, 0 : wavs_aug.shape[1]] = wavs_aug
#         wavs_aug = zero_sig
#     return wavs_aug

In [ ]:
# x = ds[1211]['sig'].unsqueeze(0)
# x1 = compute_aug(x)
# x2 = compute_aug(x)

In [ ]:
# sound(x[0].numpy(), 16000, label='orig')
# sound(x1[0].numpy(), 16000, label='x1')
# sound(x2[0].numpy(), 16000, label='x2')

# Spec Augment

In [ ]:
from speechbrain.lobes import features
import augmentation
compute_features = features.Fbank(
    n_mels=80,
    sample_rate=16000,
    left_frames=0,
    right_frames=0,
    deltas=False,
    n_fft=400,
    win_length=25,
    hop_length=10,
    f_min=0
)

spec_domain_aug = augmentation.TFAugmentation(
    time_warp=True,
    time_warp_window=8,
    freq_mask=True,
    freq_mask_width=(0,10),
    n_freq_mask=2,
    time_mask=True,
    time_mask_width=(0, 10),
    n_time_mask=2,
    replace_with_zero=True,
    time_roll=True,
    time_roll_limit=(0,200),
    freq_shift=True,
    freq_shift_limit=(-10, 10),
)

In [ ]:
def plot_spec_aug(x):
    '''
        x: (t,)
    '''
    import matplotlib.pyplot as plt
    spec = compute_features(x.unsqueeze(0))
    aug = spec_domain_aug(spec, None)
    plt.subplot(1, 2, 1)
    plt.pcolormesh(spec.squeeze(0).numpy().T)
    plt.subplot(1, 2, 2)
    plt.pcolormesh(aug.squeeze(0).numpy().T)

plot_spec_aug(x1)